@Joshua Ong

In [1]:
using Pkg
Pkg.add("Pkg")
Pkg.add("Dates")

joshpath = "C:/Users/Owner/Documents/Austin/AmbulanceDeployment/AmbulanceDeployment.jl-legacy"

PROJECT_ROOT = joshpath
currentpath = joshpath

cd(currentpath)
Pkg.activate(".")

#include(".//src//problem.jl")
#include(".//src//model.jl")

using AmbulanceDeployment 
using DataFrames, Winston, JLD, CSV, Gurobi, JuMP, GLPK, Dates

   Updating registry at `C:\Users\Owner\.julia\registries\General`
  Resolving package versions...
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Manifest.toml`
  Resolving package versions...
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Manifest.toml`
 Activating environment at `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`
┌ Info: Precompiling AmbulanceDeployment [d2c08a37-75f3-4934-bcbc-64feba624d07]
└ @ Base loading.jl:1278
┌ Warning: Package AmbulanceDeployment does not have Random in its dependencies:
│ - If you have AmbulanceDeployment checked out for development and have
│   added Random as a dependency but hav

In [3]:
# hourly calls - regions x hours x number of calls/ per region per hour (e.g. 210 x 10000 x Z)
#adjacent_nbhd - boolean matrix, regionxregion (210x210) - if neighborhoods/region are adjacent?
#coverage - boolean matrix, regions x stations (210x44) - true if station is within coverage of region
#incidents - god object
hourly_calls = CSV.File(PROJECT_ROOT * "/test/austin-data/Full_WeekdayCalls.csv") |> DataFrame
# weekend_hourly_calls = CSV.File("data/processed/2-weekend_calls.csv") |> DataFrame
adjacent_nbhd = CSV.File(PROJECT_ROOT * "/test/austin-data/adjacent_nbhd.csv") |> DataFrame
coverage = CSV.read(PROJECT_ROOT * "/test/austin-data/coverage_real.csv", DataFrame, header=false)
coverage = convert(Array{Bool, 2}, coverage[:, :])
#incidents = CSV.File(PROJECT_ROOT * "/test/austin-data/austin_incidents.csv") |> DataFrame

210×44 Array{Bool,2}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  1  0
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1    

In [5]:
#regions - list of regions/neighborhoods where demands come from (1 ... 210)
#locations -list of stations (1 ... 44)
#demands - hours x regions = demands
regions = Int[parse(Int,string(x)) for x in names(hourly_calls[:,6:ncol(hourly_calls)])]
locations = collect(1:size(coverage,2))
adjacent = convert(Array, adjacent_nbhd[:,2:ncol(adjacent_nbhd)])[regions,regions] .> 0.5
demand = convert(Array,hourly_calls[:,6:end]);

In [7]:
# We focus on emergency calls during the "peak period" (8AM - 8PM),
# with the emergency calls from the first 3 month as our training set,
# and the subsequent emergency calls from the remaining months as our test set
peak_period = (hourly_calls[!,:hour] .>= 8) .* (hourly_calls[!,:hour] .<= 20)
indices = 1:DataFrames.nrow(hourly_calls);
train_filter = (hourly_calls[!,:year] .== 2019) .* (hourly_calls[!,:month] .<= 3)
test_filter  = .~train_filter;
train_indices = indices[train_filter]
test_indices = indices[test_filter];

In [8]:
# Same as for the hourly calls; but this is for individual emergency calls
# inc_peak_period = (calls[!,:hour] .>= 8) .* (calls[!,:hour] .<= 20)
# inc_indices = 1:DataFrames.nrow(calls);

# inc_train_filter = (calls[!,:year] .== 2019) .* (calls[!,:month] .<= 3)
# inc_test_filter  = .~inc_train_filter

# inc_train_indices = inc_indices[inc_train_filter]
# inc_test_indices = inc_indices[inc_test_filter];

In [11]:
# we distinguish between peak and offpeak hours
train_peak = indices[peak_period .* train_filter]
train_offpeak = indices[.~peak_period .* train_filter]

test_peak = indices[peak_period .* test_filter]
test_offpeak = indices[.~peak_period .* test_filter]

p = DeploymentProblem(30, length(locations), length(regions), demand, train_indices,
      test_indices, coverage[regions,:], Array{Bool,2}(adjacent));

#test_inc_peak = inc_peak_period .* inc_test_filter
#test_inc_offpeak = .~inc_peak_period .* inc_test_filter;

In [14]:
#data structures to record results
scenarios = Dict{Symbol, Dict{Int, Vector{Vector{Int}}}}()
generated_deployment = Dict{Symbol, Dict{Int, Vector{Vector{Int}}}}()
upperbounds = Dict{Symbol, Dict{Int, Vector{Float64}}}()
lowerbounds = Dict{Symbol, Dict{Int, Vector{Float64}}}()
upptiming = Dict{Symbol, Dict{Int, Vector{Float64}}}()
lowtiming = Dict{Symbol, Dict{Int, Vector{Float64}}}()
amb_deployment = Dict{Symbol, Dict{Int, Vector{Int}}}()

Dict{Symbol,Dict{Int64,Array{Int64,1}}}()

In [20]:
(deployment_model, name) = (next_dp -> RobustDeployment(next_dp, α=0.1), :Robust01)
amb_deployment[name] = Dict{Int, Vector{Int}}()
scenarios[name] = Dict{Int, Vector{Vector{Int}}}()
generated_deployment[name] = Dict{Int, Vector{Vector{Int}}}()
upperbounds[name] = Dict{Int, Vector{Float64}}()
lowerbounds[name] = Dict{Int, Vector{Float64}}()
upptiming[name] = Dict{Int, Vector{Float64}}()
lowtiming[name] = Dict{Int, Vector{Float64}}()
namb = 50
println("$namb ")
p.nambulances = namb


50 


50

In [21]:
model = deployment_model(p)
set_optimizer(model.m, Gurobi.Optimizer)
@time AmbulanceDeployment.optimize!(model, p);

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Optimize a model with 254 rows, 45 columns and 5858 nonzeros
Variable types: 1 continuous, 44 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+01]

Loaded MIP start with objective 0


Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
  0.003798 seconds (16.41 k allocations: 857.992 KiB)


In [23]:
amb_deployment[name][namb] = deployment(model)

# for tracking purposes
scenarios[name][namb] = model.scenarios
generated_deployment[name][namb] = model.deployment
upperbounds[name][namb] = model.upperbounds
lowerbounds[name][namb] = model.lowerbounds
upptiming[name][namb] = model.upptiming
lowtiming[name][namb] = model.lowtiming

Float64[]

In [27]:
model.upperbounds[0]

LoadError: [91mBoundsError: attempt to access 0-element Array{Float64,1} at index [0][39m